<a href="https://colab.research.google.com/github/kgaurav1991/Python-DataAnalysis/blob/main/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

full = pd.concat([train, test], sort=False)

# Title
full['Title'] = full['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
full['Title'] = full['Title'].replace(['Mlle','Ms'],'Miss')
full['Title'] = full['Title'].replace('Mme','Mrs')
full['Title'] = full['Title'].replace(
    ['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],
    'Rare'
)

# Family Features
full['FamilySize'] = full['SibSp'] + full['Parch'] + 1
full['IsAlone'] = (full['FamilySize']==1).astype(int)

# Fare per person
full['Fare'] = full['Fare'].fillna(full['Fare'].median())
full['FarePerPerson'] = full['Fare'] / full['FamilySize']

# Age
full['Age'] = full['Age'].fillna(full['Age'].median())

# Embarked
full['Embarked'] = full['Embarked'].fillna(full['Embarked'].mode()[0])

# Cabin Deck
full['Deck'] = full['Cabin'].astype(str).str[0]
full['Deck'] = full['Deck'].replace('n','Unknown')

# Encode
for col in ['Sex','Embarked','Title','Deck']:
    le = LabelEncoder()
    full[col] = le.fit_transform(full[col])

full = full.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

train_df = full[:len(train)]
test_df = full[len(train):]

X = train_df.drop("Survived", axis=1)
y = train_df["Survived"]
X_test = test_df.drop("Survived", axis=1)

# Models
xgb = XGBClassifier(n_estimators=400, learning_rate=0.03, max_depth=4)
rf = RandomForestClassifier(n_estimators=400, max_depth=6)
lr = LogisticRegression(max_iter=1000)

# Train
xgb.fit(X,y)
rf.fit(X,y)
lr.fit(X,y)

# Ensemble
pred = (xgb.predict(X_test) + rf.predict(X_test) + lr.predict(X_test)) / 3
final_pred = (pred > 0.5).astype(int)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": final_pred
})

submission.to_csv("submission.csv", index=False)

<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-343434960.py:16: SyntaxWarning: invalid escape sequence '\.'
  full['Title'] = full['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
